# Smart Car Buyer AI Agent

## Overview

This notebook presents an intelligent car-buying assistant using LangGraph and an LLM model. The system assists users in defining their car-buying needs, refines search filters, and retrieves relevant listings, providing a streamlined buying experience.

In [ ]:
# Import necessary libraries
from typing import TypedDict, Dict, List, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langsmith import Client
from langsmith import traceable
from langsmith.wrappers import wrap_openai
import openai
import asyncio
from importnb import Notebook

import os
from dotenv import load_dotenv

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

with Notebook():
    from scrapers.autotrader import AutotraderInterface, WebsiteInterface

In [ ]:
!playwright install
!patchright install chromium

In [ ]:
# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "car_buyer_agent"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Initialize the LangSmith client
langsmith_client = Client()

openai_client = wrap_openai(openai.Client())

In [ ]:
class State(TypedDict):
    """Represents the state of the car-buying process."""
    user_needs: str
    web_interfaces: List[WebsiteInterface]
    listings: List[Dict[str, str]]
    selected_listing: Dict[str, str]
    additional_info: Dict[str, str]
    user_input: str
    next_node: str

In [ ]:
def parse_response(response_content: str) -> dict:
    """
    Parse the response content from the model into a dictionary format.
    
    Args:
        response_content (str): The text response from the model.
        
    Returns:
        dict: Parsed information as a dictionary.
    """
    parsed_info = {}
    # Simple parsing based on line-by-line key-value pairs (customize as needed)
    lines = response_content.splitlines()
    for line in lines:
        if ":" in line:
            key, value = line.split(":", 1)
            parsed_info[key.strip().lower()] = value.strip()
    return parsed_info

In [ ]:
def ask_user_needs(state: State) -> State:
    """Ask user initial questions to define their needs for the car."""

    # Check if we already have some user needs to preserve
    existing_needs = state.get("user_needs", "")
    if existing_needs:
        existing_needs_text = f"Here's what I know about your needs so far:\n{existing_needs}\n"
    else:
        existing_needs_text = "I don't have any details about your needs yet."

    # Generate the initial prompt for the LLM to ask the user about their car needs
    initial_user_needs_prompt = (
        f"{existing_needs_text}\n\n"
        "Please tell me more about your requirements to help find the best car for you. "
        "Could you share details like budget, usage (e.g., daily commute, off-road), "
        "preferred fuel type, and any must-have features?"
    )
    
    print(initial_user_needs_prompt)

    user_input = input("Please enter your response: ")

    # Construct a follow-up prompt to include both the user input and any existing needs
    followup_prompt = (
        "Here's the information provided:\n\n" +
        (f"Existing needs:\n{existing_needs_text}\n\n" if existing_needs else "") +
        f"User input: {user_input}\n\n"
        "Based on all the provided information, please summarize the user's car-buying needs in one or a few clear, concise sentences."
    )

    # Send follow-up to the LLM to get a structured summary of user needs
    followup_response = llm(followup_prompt)
    state["user_needs"] = followup_response.content.strip()  # Store as a single string
    
    print("I have summarized your car-buying needs as follows:")
    print(state["user_needs"])

    return state

In [ ]:
def parse_user_input(state: State) -> State:
    """Parse the user's input to determine the next action."""
    user_input = state.get("user_input", "").lower()
    if "refine" in user_input:
        state["next_node"] = "build_filters"
    elif "view" in user_input:
        state["next_node"] = "fetch_additional_info"
    elif "select" in user_input:
        state["next_node"] = END
    else:
        state["next_node"] = "ask_user_needs"
    return state

In [ ]:
def build_filters(state: State) -> State:
    """Build and refine search filters based on user needs."""
    
    for interface in state["web_interfaces"]:
        filters_info = interface.get_filters_info()
        
        # TODO: Check if this website is useful to the user based on the filters
        # If not continue to the next interface
        
        # If the website is useful, use LLM to setup the filters based on user needs
        
        
        # Define system instructions with filters information
        system_message = SystemMessagePromptTemplate.from_template(
            "{filters_info}"
        )

        # Define user input prompt
        human_message = HumanMessagePromptTemplate.from_template(
            "User needs: {user_needs}"
        )

        # Create a structured prompt template
        prompt_template = ChatPromptTemplate.from_messages([system_message, human_message])

        # Format the prompt with dynamic variables
        prompt = prompt_template.format_messages(
            user_needs=state["user_needs"],
            filters_info=filters_info,
        )

        # Use the LLM to process the user's needs and set the filters
        try:
            result = llm.invoke(prompt)
            llm_response = result.content.strip()

            # Validate and set the filters for the interface
            interface.set_filters_from_llm_response(llm_response)
            print(f"Successfully set filters for: {interface.base_url}")
            print(f"Updated URL: {interface.url}")
        except ValueError as e:
            print(f"Failed to set filters for {interface.base_url}: {e}")
        except Exception as e:
            print(f"An error occurred while processing filters for {interface.base_url}: {e}")
    
    return state

In [ ]:
async def fetch_listings_from_sources(web_interfaces: List[WebsiteInterface]) -> List[Dict[str, str]]:
    """Simulate retrieval of car listings from LaCentrale and mobile.de based on filters.
    
    Args:
        filters (dict): Dictionary containing search filters (e.g., budget, fuel type).
        
    Returns:
        list: A list of dictionaries, each representing a car listing.
    """
    listings = []
    for interface in web_interfaces:
        listings += await interface.crawl()
        
    return listings

In [ ]:
def search_listings(state: State) -> State:
    """Search for cars on LaCentrale and mobile.de based on filters."""
    """Synchronous wrapper for search_listings."""
    async def _search_listings():
        return await fetch_listings_from_sources(state["web_interfaces"])
    
    listings = asyncio.run(_search_listings())
    state["listings"] = listings
    
    print(f"Successfully fetched {len(listings)} listings from the sources.")
    
    print("Here is what I found:")
    for i, listing in enumerate(listings):
        print(f"{i + 1}. {listing}")
    
    return state

In [ ]:
def display_listings(state: State) -> State:
    """Display retrieved listings and ask the user for the next action."""
    print("Here are the cars that match your requirements:")
    for i, listing in enumerate(state["listings"], 1):
        print(f"{i}.")
        for key, value in listing.items():
            formatted_key = key.replace("_", " ").capitalize()
            print(f"   {formatted_key}: {value}")
        print()  # Add an extra line for readability
    
    # Prompt user for actions
    state["user_input"] = input(
        "Would you like to refine filters, view more details, or select a car? (refine/view/select): "
    ).strip().lower()
    return state

In [ ]:
def fetch_additional_info(state: State) -> State:
    """Fetch more details about the selected car listing."""
    listing_index = int(input("Enter the number of the listing you want to view details for: ")) - 1
    state["selected_listing"] = state["listings"][listing_index]
    listing = state["selected_listing"]
    prompt = (
        f"Provide additional information about this car: {listing['title']}, "
        f"including engine specifications, common issues with this model, and market value."
    )
    result = llm(prompt)
    state["additional_info"] = parse_response(result.content)
    return state

In [ ]:
# Initialize the StateGraph
workflow = StateGraph(State)

# Define the nodes in the graph
workflow.add_node("ask_user_needs", ask_user_needs)
workflow.add_node("build_filters", build_filters)
workflow.add_node("search_listings", search_listings)
workflow.add_node("display_listings", display_listings)
workflow.add_node("fetch_additional_info", fetch_additional_info)
workflow.add_node("parse_user_input", parse_user_input)

# Define edges
workflow.add_edge("ask_user_needs", "build_filters")
workflow.add_edge("build_filters", "search_listings")
workflow.add_edge("search_listings", "display_listings")
workflow.add_edge("display_listings", "parse_user_input")
workflow.add_conditional_edges("parse_user_input", lambda state: state["next_node"])

# Set the entry and exit points
workflow.set_entry_point("ask_user_needs")
workflow.add_edge("fetch_additional_info", END)

# Compile the workflow
app = workflow.compile()

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
# Verify initial setup and function invocation
def run_car_buyer_agent():
    """Run the car-buying assistant with LangGraph."""
    initial_state = State(
        user_needs={}, 
        web_interfaces=[AutotraderInterface()], 
        listings=[], 
        selected_listing={}, 
        additional_info={},
        user_input="",
        next_node=""
    )
    result = app.invoke(initial_state)
    return result

# Execute the agent
car_buyer_result = run_car_buyer_agent()

# Print result for debugging purposes
print("Car Buyer Result:", car_buyer_result)

In [ ]:
# Display summary of the final recommendation
if "selected_listing" in car_buyer_result:
    listing = car_buyer_result["selected_listing"]
    print(f"\nFinal Recommendation:\n{listing['title']} - {listing['price']} - {listing['mileage']} km")
    print("Additional Information:")
    for key, value in car_buyer_result["additional_info"].items():
        print(f"{key}: {value}")
else:
    print("No car listing selected.")